In [ ]:
# drive mount

In [ ]:
# !pip install geojson geopandas osmnx spektral matplotlib==3.1.3

# from google.colab import files
# from google.colab import drive

# drive.mount('/content/drive')

In [ ]:
# general import

In [ ]:
!pip install spektral 

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler

import gc
import time
import seaborn as sns; sns.set()

import os 
# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
#pd.options.display.mpl_style = 'default'
from datetime import datetime
import numpy as np
from datetime import datetime, timedelta
from pytz import timezone
from dateutil import tz
import geojson
import geopandas as gpd
from fiona.crs import from_epsg
import os, json
from shapely.geometry import shape, Point, Polygon, MultiPoint
%matplotlib inline
import matplotlib.pyplot as plt
from geopandas.tools import sjoin

import matplotlib.cm as cm

import folium
import shapely.geometry

from branca.colormap import  linear
import json
import branca.colormap as cm
import matplotlib.colors as colors
%matplotlib inline

import networkx as nx
import pickle

import osmnx as ox
ox.config(log_console=True, use_cache=True)
ox.__version__

import matplotlib.colors as mcolors
import gc

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from spektral.layers import GCNConv, ChebConv, GlobalSumPool, GlobalAvgPool #(channels, K=1)


from spektral.utils import gcn_filter
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [ ]:
# for reproducibility

In [ ]:
from numpy.random import seed

# Reproducability
def set_seed(seed=31415):
    
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    
set_seed(31415)

In [ ]:
# imporing files

In [ ]:
path_graph = '../input/graph-results/graph_adj_bxl.pkl'

path_feat_flow = '../input/graph-results/FEATURES_flow_bxl.csv'
path_feat_vel = '../input/graph-results/FEATURES_vel_bxl.csv'

with open(path_graph,'rb') as f:
    graph, adj_matrix, edges, G = pickle.load(f)

adj_mx = nx.to_numpy_matrix(graph)

# flow
features_flow = pd.read_csv(path_feat_flow).iloc[:,1:].values
# vel
features_vel = pd.read_csv(path_feat_vel).iloc[:,1:].values

In [ ]:
# PARAMETERS

In [ ]:
inputs = 12
granularity = 2*2

batch_train = 64 # best 32
batch_test = 1

## split train/val/test

In [ ]:
data_flow = features_flow[:, :-1]
data_vel = features_vel[:, :-1]

data = np.concatenate([data_flow, data_vel], axis=1)

In [ ]:
# Train/test split
data_tr, data_te = data[:-168*2*granularity, :], data[-168*2*granularity:, :]

In [ ]:
# time-based Covariates

In [ ]:
datetime = features_flow[:, -1]

# for figures
print_datetime = datetime[-168*2*granularity+inputs:]

DATETIME = pd.DataFrame(datetime, columns=['Datetime'])
DATETIME['Datetime'] = pd.to_datetime(DATETIME['Datetime'])

DATETIME['minutes'] = DATETIME['Datetime'].dt.minute
DATETIME['hour'] = DATETIME['Datetime'].dt.hour

DATETIME['hour_x']=np.sin(DATETIME.hour*(2.*np.pi/23))
DATETIME['hour_y']=np.cos(DATETIME.hour*(2.*np.pi/23))

DATETIME['day'] = DATETIME['Datetime'].dt.day
DATETIME['DayOfWeek'] = DATETIME['Datetime'].dt.dayofweek

DATETIME['WorkingDays'] = DATETIME['DayOfWeek'].apply(lambda y: 2 if y < 5 else y)
DATETIME['WorkingDays'] = DATETIME['WorkingDays'].apply(lambda y: 1 if y == 5 else y)
DATETIME['WorkingDays'] = DATETIME['WorkingDays'].apply(lambda y: 0 if y == 6 else y)

DATETIME = DATETIME.drop(['Datetime','minutes','hour','day'], axis=1).values

# temporal features = 4
feat_time = 4

# datetime Train/test split
time_tr, time_te = DATETIME[:-168*2*granularity, :], DATETIME[-168*2*granularity:, :]



## scale data

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler_cov = MinMaxScaler(feature_range=(0, 1))

# fit and transform
scaled_tr = scaler.fit_transform(data_tr)
# transform
scaled_te = scaler.transform(data_te)

# fit and transform
scaled_tr_cov = scaler_cov.fit_transform(time_tr)
# transform
scaled_te_cov = scaler_cov.transform(time_te)

## prepare data for deep learning

In [ ]:
def prepare_data_DL(INPUT, FEAT, BATCH):
    
    dataset = FEAT.reshape(FEAT.shape[0], FEAT.shape[1]) 
    dataset = tf.data.Dataset.from_tensor_slices(dataset)

    inputs = dataset.window(INPUT,  shift=1,  stride=1,  drop_remainder=True)
    inputs = inputs.flat_map(lambda window: window.batch(INPUT))

    targets = dataset.window(INPUT, shift=1,  stride=1,  drop_remainder=True).skip(INPUT)
    targets = targets.flat_map(lambda window: window.batch(INPUT))

    dataset = tf.data.Dataset.zip((inputs, targets))
    dataset = dataset.batch(BATCH).prefetch(tf.data.experimental.AUTOTUNE)


    return dataset

In [ ]:
# features
loader_tr = prepare_data_DL(inputs, scaled_tr, batch_train)
loader_te = prepare_data_DL(inputs, scaled_te, batch_test)

# covariates
loader_tr_cov = prepare_data_DL(inputs, scaled_tr_cov, batch_train)
loader_te_cov = prepare_data_DL(inputs, scaled_te_cov, batch_test)

In [ ]:
l2_reg = 5e-4  # Regularization rate for l2

# Build model
class GCN_Net(Model):
    
    def __init__(self, **kwargs):
        
        super().__init__(**kwargs)

        self.street = 4524


        # GCN 1st order approximation
        self.gcn_flow_0_enc = GCNConv(12, activation="relu", kernel_regularizer=l2(l2_reg), use_bias=False)
        self.gcn_flow_1_enc = GCNConv(12, activation="relu", kernel_regularizer=l2(l2_reg), use_bias=False)

        self.gcn_vel_0_enc = GCNConv(12, activation="relu", kernel_regularizer=l2(l2_reg), use_bias=False)
        self.gcn_vel_1_enc = GCNConv(12, activation="relu", kernel_regularizer=l2(l2_reg), use_bias=False)



###################

        # encoder GRU FLOW
        # self.lstm_cells_flow_init = tf.keras.layers.GRUCell(150, activation ='relu',
        #                                 kernel_initializer='glorot_uniform',
        #                                 recurrent_initializer='glorot_uniform',
        #                                 kernel_regularizer=regularizers.l2(0.001),
        #                                 bias_initializer='zeros', dropout=0.0) 
        
        # self.lstm_flow_init = tf.keras.layers.RNN(self.lstm_cells_flow_init, return_sequences = True, return_state =True)

        self.lstm_flow_init = tf.compat.v1.keras.layers.CuDNNGRU(150, #activation ='relu',
                                        kernel_initializer='glorot_uniform',
                                        recurrent_initializer='glorot_uniform',
                                        kernel_regularizer=regularizers.l2(0.001),
                                        bias_initializer='zeros', return_sequences = True, return_state =True)

        # decoder GRU FLOW
        # self.lstm_cells_flow_fin = tf.keras.layers.GRUCell(150, activation ='relu',
        #                                 kernel_initializer='glorot_uniform',
        #                                 recurrent_initializer='glorot_uniform',
        #                                 kernel_regularizer=regularizers.l2(0.001),
        #                                 bias_initializer='zeros', dropout=0.0) 
        
        # self.lstm_flow_fin = tf.keras.layers.RNN(self.lstm_cells_flow_fin)

        self.lstm_flow_fin = tf.compat.v1.keras.layers.CuDNNGRU(150, #activation ='relu',
                                        kernel_initializer='glorot_uniform',
                                        recurrent_initializer='glorot_uniform',
                                        kernel_regularizer=regularizers.l2(0.001),
                                        bias_initializer='zeros')



        self.drop_flow = tf.keras.layers.Dropout(0.1)

        self.dense_flow_fin = tf.keras.layers.Dense(self.street*12,
                                           kernel_regularizer=regularizers.l2(0.001))
        
        

##########################

        # encoder GRU VEL
        # self.lstm_cells_vel_init = tf.keras.layers.GRUCell(150, activation ='relu',
        #                                 kernel_initializer='glorot_uniform',
        #                                 recurrent_initializer='glorot_uniform',
        #                                 kernel_regularizer=regularizers.l2(0.001),
        #                                 bias_initializer='zeros', dropout=0.0) 
        
        # self.lstm_vel_init = tf.keras.layers.RNN(self.lstm_cells_vel_init, return_sequences = True, return_state =True)

        self.lstm_vel_init = tf.compat.v1.keras.layers.CuDNNGRU(150, #activation ='relu',
                                        kernel_initializer='glorot_uniform',
                                        recurrent_initializer='glorot_uniform',
                                        kernel_regularizer=regularizers.l2(0.001),
                                        bias_initializer='zeros', return_sequences = True, return_state =True)


        # decoder GRU VEL
        # self.lstm_cells_vel_fin = tf.keras.layers.GRUCell(150, activation ='relu',
        #                                 kernel_initializer='glorot_uniform',
        #                                 recurrent_initializer='glorot_uniform',
        #                                 kernel_regularizer=regularizers.l2(0.001),
        #                                 bias_initializer='zeros', dropout=0.0) 
        
        # self.lstm_vel_fin = tf.keras.layers.RNN(self.lstm_cells_vel_fin)


        self.lstm_vel_fin = tf.compat.v1.keras.layers.CuDNNGRU(150, #activation ='relu',
                                        kernel_initializer='glorot_uniform',
                                        recurrent_initializer='glorot_uniform',
                                        kernel_regularizer=regularizers.l2(0.001),
                                        bias_initializer='zeros')



        self.drop_vel = tf.keras.layers.Dropout(0.1)


        self.dense_vel_fin = tf.keras.layers.Dense(self.street*12,
                                           kernel_regularizer=regularizers.l2(0.001))
        
        

        
##################################

        self.attention_flow = tf.keras.layers.MultiHeadAttention(num_heads=2, key_dim=2)
        self.attention_vel = tf.keras.layers.MultiHeadAttention(num_heads=2, key_dim=2)


        self.dense_temporal_0 = tf.keras.layers.Dense(150, activation ='relu') #tf.keras.layers.TimeDistributed()
        self.dense_temporal_1 = tf.keras.layers.Dense(64, activation ='relu')
        self.dense_temporal_2 = tf.keras.layers.Dense(150)


        self.drop = tf.keras.layers.Dropout(0.1)   

        self.add = tf.keras.layers.Add()
        self.norm = tf.keras.layers.LayerNormalization()

        self.reshape = tf.keras.layers.Reshape([12, self.street])
        self.repeat = tf.keras.layers.RepeatVector(12)
        

    def call(self, flow_vel, a, past_cov, fut_cov):

        # sparse matrix
        sparse_a = tf.sparse.from_dense(a)
  
        # flow
        flow_0 = flow_vel[:, :, :-self.street] 
        # velocity
        vel_0  = flow_vel[:, :, -self.street:]
        

        # shape for gcn
        flow_sh = tf.reshape(flow_0 , [flow_0.shape[0], flow_0 .shape[2], flow_0.shape[1]])
        vel_sh = tf.reshape(vel_0 , [vel_0.shape[0], vel_0.shape[2], vel_0.shape[1]])

        # two gcn on flow and vel
        gcn_flow = self.gcn_flow_0_enc([flow_sh, sparse_a])
        gcn_flow = self.gcn_flow_1_enc([gcn_flow, sparse_a])

        gcn_vel = self.gcn_vel_0_enc([vel_sh, sparse_a])
        gcn_vel = self.gcn_vel_1_enc([gcn_vel, sparse_a])
        
        # shape for lstm
        flow_sh = tf.reshape(gcn_flow, [gcn_flow.shape[0], gcn_flow.shape[2], gcn_flow.shape[1]])
        vel_sh = tf.reshape(gcn_vel, [gcn_vel.shape[0], gcn_vel.shape[2], gcn_vel.shape[1]])

        
        # two lstm models
        flow_init, h_flow = self.lstm_flow_init(flow_sh)
        vel_init, h_vel = self.lstm_vel_init(vel_sh)


        # merge layer
        # output
        flow_vel_i = tf.concat([flow_init, vel_init], axis=2) # 

        flow_vel = self.dense_temporal_0(flow_vel_i)
        flow_vel = self.dense_temporal_1(flow_vel)
        flow_vel_f = self.dense_temporal_2(flow_vel)


        # multi-head attention
        # # Q: flow & vel
        # # K: flow
        att_flow, weight_flow = self.attention_flow(flow_vel_f, flow_init,
                               return_attention_scores=True)


        # # Q: vel & vel
        # # K: vel
        att_vel, weight_vel = self.attention_vel(flow_vel_f, vel_init,
                               return_attention_scores=True)

# -----

        # concatenate covariates
        # output flow_vel - covariates
        fut_cov = tf.cast(fut_cov, dtype=tf.float32)
        concat_flow = tf.concat([att_flow, fut_cov], axis=2)
        concat_vel = tf.concat([att_vel, fut_cov], axis=2)
  
        # two models for flow and speed respectively

        # flow
        flow_final = self.lstm_flow_fin(concat_flow , initial_state = [flow_vel_f[:,-1,:]]) # , initial_state = [flow_vel_f]) #  flow_vel_f, h_flow flow_vel_f[:,-1,:]
        flow = self.drop_flow(flow_final)
        flow = self.dense_flow_fin(flow)
        flow = self.reshape(flow)


        # velocity
        vel_final = self.lstm_vel_fin(concat_vel, initial_state = [flow_vel_f[:,-1,:]]) # , initial_state = [flow_vel_f]) 
        vel = self.drop_vel(vel_final)
        vel = self.dense_vel_fin(vel)
        vel = self.reshape(vel)


        # concatenate two finals results
        final = tf.concat([flow, vel], axis=-1)

        return final, weight_flow, weight_vel


# Create model
model = GCN_Net()
optimizer = Adam(lr=0.001)
loss_fn = tf.keras.losses.MeanAbsoluteError()


In [ ]:
adj_matrix = gcn_filter(adj_matrix, symmetric=True)

In [ ]:
del data_tr, data_te, data_flow, data_vel, data, scaled_tr, scaled_te
gc.collect()

In [ ]:
# Training function
@tf.function
def train_on_batch(inputs, target, past_cov, fut_cov):
    
    loss = 0

    with tf.GradientTape() as tape:
        
        predictions, weight_flow, weight_vel = model(inputs, adj_matrix, past_cov, fut_cov, training=True) #predictions
        
        loss = loss_fn(target, predictions)


    variables = model.trainable_variables 

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))
    
    return loss

In [ ]:
EPOCHS = 250

start = time.time()
    
# Keep results for plotting
train_loss_results = []

samples_cov = list(loader_tr_cov)

for epoch in range(EPOCHS):
    
    epoch_loss_avg = tf.keras.metrics.Mean()
     
    step = 0

    for batch in loader_tr:

        cov = samples_cov[step]
        past_cov = cov[0]
        fut_cov = cov[1]
        
        # Training step
        inputs, target = batch
        
        loss = train_on_batch(inputs, target, past_cov, fut_cov)
        
        # Track progress
        epoch_loss_avg.update_state(loss)

        step+=1

    # End epoch
    train_loss_results.append(epoch_loss_avg.result())

    if epoch % 10 == 0:
            print("Epoch {}: Loss MAE: {:.5f}".format(epoch, epoch_loss_avg.result()))
        
print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [ ]:
fig, axes = plt.subplots(1, sharex=True, figsize=(12, 8))
fig.suptitle('Training Metrics')

axes.set_ylabel("Loss (MAE)", fontsize=14)
axes.plot(train_loss_results)
axes.set_xlabel("Epoch", fontsize=14)
plt.show()

In [ ]:
def inverse_transform(forecasts, scaler):
    # invert scaling
    inv_pred = scaler.inverse_transform(forecasts)
    return inv_pred

In [ ]:
def evaluate_forecasts(targets, forecasts, n_seq):
    
    list_rmse = []
    list_mae = []
    
    for i in range(n_seq):
        true = np.vstack([target[i] for target in targets])
        predicted = np.vstack([forecast[i] for forecast in forecasts])
        
        rmse = np.sqrt((np.square(true - predicted)).mean(axis=0))
        mae = np.absolute(true - predicted).mean(axis=0)
        
        list_rmse.append(rmse)
        list_mae.append(mae)
        
    list_rmse = np.vstack(list_rmse)
    list_mae = np.vstack(list_mae)
    
    return list_rmse, list_mae

In [ ]:
def plot_attention(attention):
    
    fig = plt.figure(figsize=(10,10))
    plt.matshow(attention, cmap='viridis')

    return plt.show()

In [ ]:
from tqdm import tqdm
from matplotlib import pyplot

forecasts = []
targets = []

rmse_list = []
mae_list = []

samples_cov = list(loader_te_cov)


# del loader_te_cov, loader_tr, loader_tr_cov
# gc.collect()


for (step, (inp, targ)) in enumerate(tqdm(loader_te)):
    
            
        timestamp = print_datetime[step]

        cov = samples_cov[step]
        past_cov = cov[0]
        fut_cov = cov[1]
        
        pred, weight_flow, weight_vel  = model(inp, adj_matrix, past_cov, fut_cov, training=False)
        

#         columns_sums = np.vstack(weight_flow[0]).sum(axis=0)
#         new_matrix = weight_flow[0]/columns_sums
#         plot_attention(np.mean(new_matrix, axis=0))
        
#         fig = plt.figure(figsize=(10,10))
#         pyplot.plot(range(12), np.mean(np.mean(inp[:, :, :-4524],axis=0),axis=1)) 
#         pyplot.bar(range(12), np.mean(np.mean(new_matrix, axis=0),axis=0).reshape(-1,), color ='red')
#         pyplot.plot([None for i in np.mean(np.mean(inp[:, :, :-4524], axis=0), axis=1).tolist()] + [x for x in np.mean(pred[:, :, :-4524][0], axis=1).tolist()])
#         pyplot.plot([None for i in np.mean(np.mean(inp[:, :, :-4524], axis=0), axis=1).tolist()] + [x for x in np.mean(targ[:, :, :-4524][0], axis=1).tolist()])
#         plt.show()
        

        truth = inverse_transform(targ[0],  scaler)
        pred = inverse_transform(pred[0],  scaler)       
  
        forecasts.append(pred)
        targets.append(truth)
            

        rmse, mae = evaluate_forecasts(targets, forecasts, 12)
        
        rmse_list.append(rmse)
        mae_list.append(mae)
           
        
        del pred, truth, inp, targ, fut_cov, past_cov, cov
        gc.collect()
  

In [ ]:
np.mean(rmse_list)

In [ ]:
np.mean(mae_list)

In [ ]:
RMSE_MEAN = np.mean(rmse_list,axis=0).mean(axis=1)
RMSE_STD =  np.std(rmse_list,axis=0).std(axis=1)

for i in range(len(RMSE_MEAN)):
    print('t+'+str(i+1)+' RMSE MEAN ' +str(np.round(RMSE_MEAN[i],3))+' +- '+str(np.round(RMSE_STD[i],3)))
    print('')

In [ ]:
MAE_MEAN = np.mean(mae_list,axis=0).mean(axis=1)
MAE_STD =  np.std(mae_list,axis=0).std(axis=1)

for i in range(len(MAE_MEAN)):
    print('t+'+str(i+1)+' MAE MEAN ' +str(np.round(MAE_MEAN[i],3))+' +- '+str(np.round(MAE_STD[i],3)))
    print('')

In [ ]:
with open('MTL_GCN_GRU_RMSE.pkl', 'wb') as f:  
    pickle.dump(rmse_list, f)

with open('MTL_GCN_GRU_MAE.pkl', 'wb') as f:  
    pickle.dump(mae_list, f)


In [ ]:
with open('FORECAST_ASTMTL.pkl', 'wb') as f:  
    pickle.dump([forecasts,targets], f)
